[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# **Cell Segmentation Using YOLO**

---

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog/<BLOG_NAME>)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)

## Overview

This notebook demonstrates a complete workflow for segmenting and analyzing cells in microscopy video using the YOLO (You Only Look Once) segmentation model.

#### Real-World Applications:
- Cell culture analysis  
- Automated microscopy workflows  
- Biomedical image processing  
- Quantifying cell morphology and movement over time

# COCO to YOLO Format Conversion

Converts COCO-style segmentation annotations to YOLO segmentation dataset format.  
- Requires: `annotation.json` and images in `frames_output` directory.
- Output: Generated YOLO dataset folder.
- Parameters: allows train/val split, shuffling, and verbose mode.


In [ ]:
!git clone https://github.com/Labellerr/yolo_finetune_utils.git

In [ ]:
from yolo_finetune_utils.frame_extractor import extract_random_frames

extract_random_frames(
        paths=[r"D:\Professional\Projects\Cell_Segmentation_using_YOLO\video\cell segmentation_bw_trim.mp4"],
        total_images=5,
        out_dir="frames_output",
        jpg_quality=100,
        seed=42
    )

In [ ]:
from yolo_finetune_utils.coco_yolo_converter.seg_converter import coco_to_yolo_converter

ANNOTATION_JSON = r"annotation.json"
IMAGE_DIR = r"frames_output"


coco_to_yolo_converter(
        json_path=ANNOTATION_JSON,
        images_dir=IMAGE_DIR,
        output_dir="yolo_dataset",
        use_split=False,
        train_ratio=0.8,
        val_ratio=0.2,
        test_ratio=0.0,
        shuffle=True,
        verbose=False
    )

# Clear CUDA Cache

Frees up GPU memory by clearing unused cached memory before starting new tasks or model training.


In [1]:
import torch
torch.cuda.empty_cache()

# Check GPU Memory Status

Prints the GPU memory currently allocated, cached, and free.  
Useful for diagnosing memory usage before training or inference.


In [2]:
# Check GPU memory status
print(f"Allocated: {torch.cuda.memory_allocated(0)/1024**3:.2f} GB")
print(f"Cached: {torch.cuda.memory_reserved(0)/1024**3:.2f} GB")
print(f"Free: {torch.cuda.mem_get_info(0)[0]/1024**3:.2f} GB")

Allocated: 0.00 GB
Cached: 0.00 GB
Free: 6.78 GB


# Load and Train YOLO Segmentation Model

Loads the YOLO segmentation model and trains it using the converted YOLO dataset.
- Data: Path to YOLO-style `data.yaml`
- Parameters: epochs, image size, batch size, device, dataloader workers, experiment name.


In [ ]:
from ultralytics import YOLO
# Load a model
model = YOLO("yolo11m-seg.pt")

# Train the model
results = model.train(
    data=r"yolo_dataset\data.yaml",    # Path to your dataset YAML file
    epochs=500,                        # Number of training epochs
    imgsz=640,                         # Image size
    batch=-1,                          # Batch size
    device=0,                          # GPU device (0 for first GPU, 'cpu' for CPU)
    workers=4,                         # Number of dataloader workers
    name="Model m"
)

# CellSegmentationVideo Class Definition

Defines the `CellSegmentationVideo` class for performing video cell segmentation using YOLO.
- Initialization requires YOLO model path, confidence threshold, IOU threshold.
- Contains methods for polygon area calculation, mask center localization, frame processing, and video processing.
- Tracks cells, draws masks and borders, labels areas and track IDs.


In [6]:
import cv2
import numpy as np
from collections import defaultdict
from ultralytics import YOLO

class CellSegmentationVideo:
    def __init__(self, model_path, conf_threshold=0.25, iou_threshold=0.7):
        """
        Initialize the cell segmentation video inference class.
        
        Args:
            model_path: Path to trained YOLO segmentation model
            conf_threshold: Confidence threshold for detections
            iou_threshold: IOU threshold for NMS
        """
        self.model = YOLO(model_path)
        self.conf_threshold = conf_threshold
        self.iou_threshold = iou_threshold
        
        # Generate distinct colors for different tracked objects
        np.random.seed(42)
        self.colors = [tuple(map(int, np.random.randint(0, 255, 3))) 
                       for _ in range(100)]
        
        # Track history for smoothing
        self.track_history = defaultdict(lambda: [])
        
    def calculate_polygon_area(self, points):
        """Calculate area of polygon using Shoelace formula."""
        x = points[:, 0]
        y = points[:, 1]
        return 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))
    
    def get_polygon_center(self, points):
        """Calculate center of polygon."""
        moments = cv2.moments(points)
        if moments['m00'] != 0:
            cx = int(moments['m10'] / moments['m00'])
            cy = int(moments['m01'] / moments['m00'])
            return (cx, cy)
        return (int(np.mean(points[:, 0])), int(np.mean(points[:, 1])))
    
    def process_frame(self, frame):
        """
        Process a single frame with cell segmentation.
        
        Args:
            frame: Input frame (BGR format)
            
        Returns:
            Annotated frame with colored segmentation masks and areas
        """
        # Run tracking (built-in YOLO tracker)
        results = self.model.track(
            frame, 
            persist=True,
            conf=self.conf_threshold,
            iou=self.iou_threshold,
            verbose=False
        )
        
        annotated_frame = frame.copy()
        
        if results[0].masks is not None:
            masks = results[0].masks.xy
            boxes = results[0].boxes
            
            # Get track IDs if available
            if boxes.id is not None:
                track_ids = boxes.id.int().cpu().tolist()
            else:
                track_ids = list(range(len(masks)))
            
            for mask, track_id in zip(masks, track_ids):
                if len(mask) == 0:
                    continue
                
                # Get color for this track
                color = self.colors[track_id % len(self.colors)]
                
                # Convert mask to integer coordinates
                mask_points = mask.astype(np.int32)
                
                # Draw filled polygon with transparency
                overlay = annotated_frame.copy()
                cv2.fillPoly(overlay, [mask_points], color)
                cv2.addWeighted(overlay, 0.4, annotated_frame, 0.6, 0, annotated_frame)
                
                # Draw solid border
                cv2.polylines(annotated_frame, [mask_points], True, color, 2)
                
                # Calculate area (in pixels squared)
                area = self.calculate_polygon_area(mask_points)
                
                # Get center point
                center = self.get_polygon_center(mask_points)
                
                # Display area at center
                area_text = f"{area:.0f} px sq"
                
                # Get text size for background
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.5
                thickness = 1
                (text_w, text_h), baseline = cv2.getTextSize(
                    area_text, font, font_scale, thickness
                )
                
                # Draw background rectangle for text
                cv2.rectangle(
                    annotated_frame,
                    (center[0] - text_w//2 - 5, center[1] - text_h//2 - 5),
                    (center[0] + text_w//2 + 5, center[1] + text_h//2 + 5),
                    (255, 255, 255),
                    -1
                )
                
                # Draw area text
                cv2.putText(
                    annotated_frame,
                    area_text,
                    (center[0] - text_w//2, center[1] + text_h//2),
                    font,
                    font_scale,
                    (0, 0, 0),
                    thickness
                )
                
                # Optional: Draw track ID
                cv2.putText(
                    annotated_frame,
                    f"ID:{track_id}",
                    (center[0] - text_w//2, center[1] - text_h//2 - 10),
                    font,
                    0.4,
                    color,
                    1
                )
        
        return annotated_frame
    
    def process_video(self, input_path, output_path, display=True):
        """
        Process entire video file.
        
        Args:
            input_path: Path to input video
            output_path: Path to save output video
            display: Whether to display video while processing
        """
        cap = cv2.VideoCapture(input_path)
        
        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Setup video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        frame_count = 0
        
        print(f"Processing video: {total_frames} frames at {fps} FPS")
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Process frame
            annotated_frame = self.process_frame(frame)
            
            # Write to output
            out.write(annotated_frame)
            
            # Display if requested
            if display:
                cv2.imshow('Cell Segmentation', annotated_frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            
            frame_count += 1
            if frame_count % 30 == 0:
                print(f"Processed {frame_count}/{total_frames} frames")
        
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        
        print(f"Video saved to: {output_path}")



# Specify Model Path

Specifies the file path to the trained YOLO segmentation model weights.  
Update this path as needed to match your environment.


In [7]:
model_path = r"D:\Professional\Projects\Cell_Segmentation_using_YOLO\runs\segment\Model m\weights\last.pt"

# Example Usage—Run Segmentation on a Video

Demonstrates how to use the `CellSegmentationVideo` class:
- Initializes the processor.
- Runs segmentation inference on a sample video.
- Saves the output and optionally displays annotated frames.


In [9]:
# Example usage
if __name__ == "__main__":
    # Initialize the segmentation processor
    segmentor = CellSegmentationVideo(
        model_path=model_path,
        conf_threshold=0.5,
        iou_threshold=0.3
    )
    
    # Process a video file
    segmentor.process_video(
        input_path=r"D:\Professional\Projects\Cell_Segmentation_using_YOLO\video\cell segmentation_bw.mp4",
        output_path="output_segmented_2.mp4",
        display=False
    )
    

Processing video: 423 frames at 30 FPS
Processed 30/423 frames
Processed 60/423 frames
Processed 90/423 frames
Processed 120/423 frames
Processed 150/423 frames
Processed 180/423 frames
Processed 210/423 frames
Processed 240/423 frames
Processed 270/423 frames
Processed 300/423 frames
Processed 330/423 frames
Processed 360/423 frames
Processed 390/423 frames
Processed 420/423 frames
Video saved to: output_segmented_2.mp4
